### Spark Moive Recommendation
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StringType,IntegerType, FloatType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import pyspark.sql.functions as F
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [3]:
os.environ["PYSPARK_PYTHON"] = "python3"

## Part0: Data Import

In [5]:
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [7]:
movies_df.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [8]:
ratings_df.show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

## Part I: Exploratory Data Analysis

In [10]:
tmp1 = ratings_df.groupBy('userId').count().select(F.min('count')).collect()[0][0]
print('Minimum number of ratings per user is {}'.format(tmp1))

Minimum number of ratings per user is 20

In [11]:
tmp2 = ratings_df.groupBy('movieId').count().select(F.min('count')).collect()[0][0]
print('Minimum number of ratings per movie is {}'.format(tmp2))

Minimum number of ratings per movie is 1

In [12]:
tmp1 = ratings_df.groupBy("movieId").count()
tmp1 = tmp1.filter(tmp1['count']==1).count()
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

In [13]:
tmp1 = ratings_df.groupBy("movieId").count().select(F.avg('count')).collect()[0][0]
print('Average number of ratings per movie is {0:.2f}'.format(tmp1))

Average number of ratings per movie is 10.37

In [14]:
tmp2 = ratings_df.groupBy("userId").count().select(F.avg('count')).collect()[0][0]
print('Average number of ratings per user is {0:.2f}'.format(tmp2))

Average number of ratings per user is 165.30

In [15]:
movie_num = movies_df.select('movieId').distinct().count()
user_num = ratings_df.select('userId').distinct().count()
rating_num = ratings_df.select('rating').count()
print('Total number of users is {}. '.format(user_num))
print('Total number of movies is {}. '.format(movie_num))
print('Total number of ratings is {}. '.format(rating_num))

Total number of users is 610. 
Total number of movies is 9742. 
Total number of ratings is 100836.

In [16]:
# The sparsity of the movie ratings
# sparcity = 1- rating_num/(movie_num*user_num)
denominator = movie_num*user_num
numerator = rating_num
sparsity = (1-numerator/denominator)*100
print ("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is 98.30% empty.

In [17]:
movie_num_rated = ratings_df.select('movieId').distinct().count()
print('Total number of movies rated is {}.'.format(movie_num_rated))

Total number of movies rated is 9724.

In [18]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### Movies not rated before.

In [20]:
spark.sql(
  '''
SELECT m.*,r.rating
FROM movies m 
    LEFT JOIN ratings r ON m.movieId = r.movieId
WHERE r.rating is Null
'''
).show(5)

+-------+--------------------+--------------------+------+
movieId| title| genres|rating|
+-------+--------------------+--------------------+------+
 1076|Innocents, The (1...|Drama|Horror|Thri...| null|
 2939| Niagara (1953)| Drama|Thriller| null|
 3338|For All Mankind (...| Documentary| null|
 3456|Color of Paradise...| Drama| null|
 4194|I Know Where I'm ...| Drama|Romance|War| null|
+-------+--------------------+--------------------+------+
only showing top 5 rows

### Movie Genres

Each movie belongs to more than 1 genre,as shown below. We need to seperate these genres.

In [23]:
spark.sql(
  '''
SELECT DISTINCT title, genres
FROM movies
  '''
).show(5)

+--------------------+--------------------+
 title| genres|
+--------------------+--------------------+
Color of Night (1...| Drama|Thriller|
Surviving the Gam...|Action|Adventure|...|
Little Big League...| Comedy|Drama|
 Whatever (1998)| Drama|
 Ronin (1998)|Action|Crime|Thri...|
+--------------------+--------------------+
only showing top 5 rows

In [24]:
def split_genre(x):
  return x.split('|')
sg_udf = udf(lambda x: split_genre(x),ArrayType(StringType()))

In [25]:
genres_df = spark.sql(
  '''
  SELECT DISTINCT title, genres
  FROM movies
  '''
)
genres_df = genres_df.withColumn('genres_sep',sg_udf('genres'))
genres_df.show(5)

+--------------------+--------------------+--------------------+
 title| genres| genres_sep|
+--------------------+--------------------+--------------------+
Color of Night (1...| Drama|Thriller| [Drama, Thriller]|
Surviving the Gam...|Action|Adventure|...|[Action, Adventur...|
Little Big League...| Comedy|Drama| [Comedy, Drama]|
 Whatever (1998)| Drama| [Drama]|
 Ronin (1998)|Action|Crime|Thri...|[Action, Crime, T...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

In [26]:
genres_sep_df = (genres_df\
                      .select("title",\
                              F.explode("genres_sep")\
                              .alias("genre")))
genres_sep_df.show(5)

+--------------------+---------+
 title| genre|
+--------------------+---------+
Color of Night (1...| Drama|
Color of Night (1...| Thriller|
Surviving the Gam...| Action|
Surviving the Gam...|Adventure|
Surviving the Gam...| Thriller|
+--------------------+---------+
only showing top 5 rows

We store all the movie genres in a list called movie_genres_list.

In [28]:
movie_genres_list = []
for i in genres_sep_df.select('genre').distinct().collect():
  movie_genres_list.append(i[0])
print(type(movie_genres_list))
print(movie_genres_list)

<class 'list'>
['Crime', 'Romance', 'Thriller', 'Adventure', 'Drama', 'War', 'Documentary', 'Fantasy', 'Mystery', 'Musical', 'Animation', 'Film-Noir', '(no genres listed)', 'IMAX', 'Horror', 'Western', 'Comedy', 'Children', 'Action', 'Sci-Fi']

### Movie for Each Category

In [30]:
genres_sep_df.registerTempTable('genres_sep')

In [31]:
spark.sql(
  '''
SELECT genre,count(*) AS count
FROM genres_sep
GROUP BY 1
ORDER BY 2 DESC
'''
).show(5)

+--------+-----+
 genre|count|
+--------+-----+
 Drama| 4361|
 Comedy| 3756|
Thriller| 1894|
 Action| 1828|
 Romance| 1596|
+--------+-----+
only showing top 5 rows

We put each movie under its genre. This is stored in a dictionary called movie_genre_dict, where the key is the genre, and the values is a list that contains the movies belonging to this genre.

In [33]:
movie_genre_dict = dict()
for _ in movie_genres_list:
  movie_genre_dict[_] = list()
for index, row in genres_sep_df.toPandas().iterrows():
   movie_genre_dict[row["genre"]].append(row["title"])

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

Let's take a look at the Film-Noir genre as an example.

In [35]:
print('Here are movies that belong to the Film-Noir genre:'+'\n',movie_genre_dict['Film-Noir'])

Here are movies that belong to the Film-Noir genre:
 ['Angel Heart (1987)', 'D.O.A. (1988)', "Killer's Kiss (1955)", 'Undercurrent (1946)', 'Killers, The (1946)', 'Strange Love of Martha Ivers, The (1946)', 'Night of the Hunter, The (1955)', 'Lost Highway (1997)', '13 Tzameti (2005)', 'Fury (1936)', 'Renaissance (2006)', 'Out of the Past (1947)', 'This Gun for Hire (1942)', 'Dark Passage (1947)', 'Grifters, The (1990)', 'Strangers on a Train (1951)', 'Hoodlum (1997)', 'High Sierra (1941)', 'Drive (2011)', 'Laura (1944)', 'Wrong Man, The (1956)', 'Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)', 'Key Largo (1948)', 'Crossfire (1947)', 'Devil in a Blue Dress (1995)', 'While the City Sleeps (1956)', 'Sorry, Wrong Number (1948)', 'Long Goodbye, The (1973)', 'Brick (2005)', "Miller's Crossing (1990)", 'Postman Always Rings Twice, The (1946)', 'Rififi (Du rififi chez les hommes) (1955)', 'Bullet to the Head (2012)', 'Maltese Falcon, The (1941)', 'Pickup on South Street (1953)', 'L.A. Confidential (1997)', 'Foreign Correspondent (1940)', 'Johnny Eager (1942)', 'Mildred Pierce (1945)', 'White Heat (1949)', 'Gilda (1946)', 'Call Northside 777 (1948)', 'Big Sleep, The (1946)', 'Third Man, The (1949)', 'Notorious (1946)', 'Dark City (1998)', 'Sin City (2005)', 'Suture (1993)', 'House of Games (1987)', 'Letter, The (1940)', 'Bitter Moon (1992)', 'Trou, Le (Hole, The) (Night Watch, The) (1960)', 'Tinker Tailor Soldier Spy (2011)', 'Stranger, The (1946)', 'You Only Live Once (1937)', 'Angels with Dirty Faces (1938)', 'M (1931)', 'Down by Law (1986)', 'Lady from Shanghai, The (1947)', 'Limits of Control, The (2009)', 'Suddenly (1954)', 'Mulholland Drive (2001)', 'Born to Kill (1947)', 'Chinatown (1974)', 'Killing, The (1956)', 'Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)', 'Too Late for Tears (1949)', '2 Days in the Valley (1996)', 'Sweet Smell of Success (1957)', 'Suspicion (1941)', 'In a Lonely Place (1950)', 'Double Life, A (1947)', 'Touch of Evil (1958)', 'Blood Simple (1984)', 'Pure Formality, A (Pura formalità, Una) (1994)', 'Asphalt Jungle, The (1950)', 'High and Low (Tengoku to jigoku) (1963)', 'D.O.A. (1950)', 'Harder They Fall, The (1956)', 'Detour (1945)', 'Kiss Me Deadly (1955)', 'Rubber (2010)', 'One False Move (1992)', 'I Am a Fugitive from a Chain Gang (1932)', 'This World, Then the Fireworks (1997)', 'Double Indemnity (1944)', 'Stray Dog (Nora inu) (1949)']

## Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [37]:
ratings_df.show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

In [38]:
movie_ratings_df=ratings_df.drop('timestamp')
movie_ratings_df.show(5)

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 1| 4.0|
 1| 3| 4.0|
 1| 6| 4.0|
 1| 47| 5.0|
 1| 50| 5.0|
+------+-------+------+
only showing top 5 rows

In [39]:
# Data type convert
movie_ratings_df = movie_ratings_df.withColumn("userId", movie_ratings_df["userId"].cast(IntegerType()))
movie_ratings_df = movie_ratings_df.withColumn("movieId", movie_ratings_df["movieId"].cast(IntegerType()))
movie_ratings_df = movie_ratings_df.withColumn("rating", movie_ratings_df["rating"].cast(FloatType()))

In [40]:
movie_ratings_df.show(5)

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 1| 4.0|
 1| 3| 4.0|
 1| 6| 4.0|
 1| 47| 5.0|
 1| 50| 5.0|
+------+-------+------+
only showing top 5 rows

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [42]:
#Create test and train set
(training,test)=movie_ratings_df.randomSplit([0.8,0.2],seed = 42)

## We will tune the hyperparameters using ParamGridBuilder and CrossValidator.

In [44]:
#Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",coldStartStrategy = 'drop',nonnegative = True, implicitPrefs = False)
# Confirm that a model called "als" was created
type(als)

Out[129]: pyspark.ml.recommendation.ALS

In [45]:
#Tune model using ParamGridBuilder
# We will just tune rank and regParam considering long run time, after we get the have combination, we will use larger iterations.
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [3,5,10]) \
            .addGrid(als.maxIter, [10]) \
            .addGrid(als.regParam, [0.05,0.15,0.25]) \
            .build()
print ("Num models to be tested: ", len(param_grid))

Num models to be tested: 9

In [46]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 

In [47]:
# Build Cross validation 
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

In [48]:
#Fit ALS model to training data
model = cv.fit(training)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [49]:
#Extract best model from the tuning exercise using ParamGridBuilder
best_model = model.bestModel

In [50]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [51]:
#Print evaluation metrics and model parameters
print ("**Best Model**")
print ("RMSE = "+str(rmse))
print (" Rank:",best_model._java_obj.parent().getRank())   #parent()method will return an estimator,you can get the best params then
print (" MaxIter:",best_model._java_obj.parent().getMaxIter())
print (" RegParam:",best_model._java_obj.parent().getRegParam()) 

**Best Model**
RMSE = 0.8736197761693623
 Rank: 10
 MaxIter: 10
 RegParam: 0.15

### Increase iteration number and Model testing

In [53]:
#Increase the interation for the best ALS model
#coldStartStrategy = 'drop' is important, otherwise, you will recieve rmse = nan
#Spark allows users to set the coldStartStrategy parameter to “drop” in order to drop any rows in the DataFrame of predictions that contain NaN values. 
als_50 = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",rank = 10, maxIter = 50,regParam = 0.15, nonnegative = True, coldStartStrategy = 'drop',implicitPrefs = False)
#fit the model to training data
best_model_50 = als_50.fit(training)

In [54]:
#generate predictions on test data
prediction_50 = best_model_50.transform(test)

In [55]:
#tell spark how to evaluate predictions
evaluator_50 = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
#obtain rmse
rmse_50 = evaluator_50.evaluate(prediction_50)
#print rmse
print('RMSE=',rmse_50)

RMSE= 0.8705602687772467

In [56]:
prediction_50_pd_df = prediction_50.toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [57]:
fig = plt.figure(figsize=(7, 5))
sns.scatterplot(x='movieId',y='rating', data=prediction_50_pd_df,label='rating',marker = 'o',s=20)
sns.scatterplot(x='movieId',y='prediction', data=prediction_50_pd_df,label='prediction',marker ='+',s=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel("MovieId", fontsize=14)
plt.ylabel("Rating", fontsize=14)
plt.title("Prediction vs movieId", fontsize=14)
plt.legend(loc='best', fontsize=10)
display()

In [58]:
# round the prediction to a scale of 1-5
def round_to_5scale(x):
  if x<=round(x) and 0.5<=x<=5:
    return round(x)-0.5 if round(x)-x>0.25 else round(x)
  if x>round(x) and 0.5<=x<=5:
    return round(x)+0.5 if x-round(x)>0.25 else round(x)
  if x>5:
    return 5
  if x<0.5:
    return 0.5

In [59]:
prediction_50_pd_df['prediction'] = prediction_50_pd_df['prediction'].apply(lambda x: round_to_5scale(x))

In [60]:
#After converting to a rate of 0-5
fig, ax = plt.subplots(figsize=(7, 5))
sns.scatterplot(x='movieId',y='rating', data=prediction_50_pd_df,label='rating',marker = 'o',s=20,ax=ax)
sns.scatterplot(x='movieId',y='prediction', data=prediction_50_pd_df,label='prediction',marker = '+',s=20,ax=ax)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel("MovieId", fontsize=14)
plt.ylabel("Rating", fontsize=14)
plt.title("Prediction vs movieId", fontsize=14)
plt.legend(loc='best', fontsize=10)
display()

In [61]:
fig, ax = plt.subplots(figsize=(7,5))
sns.scatterplot(x='userId',y='rating', data=prediction_50_pd_df,label='rating',marker = 'o',s=20,ax=ax)
sns.scatterplot(x='userId',y='prediction', data=prediction_50_pd_df,label='prediction',marker = '+',s=20,ax=ax)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel("UserId", fontsize=14)
plt.ylabel("Rating", fontsize=14)
plt.title("Prediction vs UserId", fontsize=14)
plt.legend(loc='best', fontsize=10)
display()

### Apply model to all data and see the performance

In [63]:
#Best_model RMSE
alldata=best_model.transform(movie_ratings_df)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6975433153657589

In [64]:
#Best_model_50 RMSE
alldata=best_model_50.transform(movie_ratings_df)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6874073412892819

In [65]:
alldata.registerTempTable("alldata")

In [66]:
spark.sql(
  '''
SELECT *
FROM movies
	LEFT JOIN alldata ON movies.movieId = alldata.movieId
  '''
).show(5)

+-------+--------------------+------+------+-------+------+----------+
movieId| title|genres|userId|movieId|rating|prediction|
+-------+--------------------+------+------+-------+------+----------+
 148|Awfully Big Adven...| Drama| 191| 148| 5.0| 4.855351|
 471|Hudsucker Proxy, ...|Comedy| 133| 471| 4.0| 2.416814|
 471|Hudsucker Proxy, ...|Comedy| 597| 471| 2.0| 3.3838356|
 471|Hudsucker Proxy, ...|Comedy| 385| 471| 4.0| 2.9433992|
 471|Hudsucker Proxy, ...|Comedy| 436| 471| 3.0| 3.3313468|
+-------+--------------------+------+------+-------+------+----------+
only showing top 5 rows

In [67]:
alldata_pd_df = alldata.toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [68]:
alldata_pd_df['prediction'] = alldata_pd_df['prediction'].apply(lambda x: round_to_5scale(x))

In [69]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.scatterplot(x='userId',y='rating', data=alldata_pd_df,label='rating',marker = 'o',s=20,ax=ax)
sns.scatterplot(x='userId',y='prediction', data=alldata_pd_df,label='prediction',marker = '+',s=20,ax=ax)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel("MovieId", fontsize=14)
plt.ylabel("Rating", fontsize=14)
plt.title("Prediction vs MovieId", fontsize=14)
plt.legend(loc='best', fontsize=10);
display()

In [70]:
fig, ax = plt.subplots(figsize=(7,5))
sns.scatterplot(x='userId',y='rating', data=alldata_pd_df,label='rating',marker = 'o',s=20,ax=ax)
sns.scatterplot(x='userId',y='prediction', data=alldata_pd_df,label='prediction',marker = '+',s=20,ax=ax)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel("UserId", fontsize=14)
plt.ylabel("Rating", fontsize=14)
plt.title("Prediction vs UserId", fontsize=14)
plt.legend(loc='best', fontsize=10);
display()

### Let's look more closely on the user and movie data to decide which user to recommend.

In [72]:
ratings_info_df = movie_ratings_df.groupBy('movieId').avg('rating')
movie_ratings_count= movie_ratings_df.groupBy('movieId').count()
ratings_info_df = ratings_info_df.join(movie_ratings_count,'movieId','left').join(movies_df,'movieId','left')
ratings_info_df.show(5)

+-------+-----------------+-----+--------------------+--------------------+
movieId| avg(rating)|count| title| genres|
+-------+-----------------+-----+--------------------+--------------------+
 1580|3.487878787878788| 165|Men in Black (a.k...|Action|Comedy|Sci-Fi|
 2366| 3.64| 25| King Kong (1933)|Action|Adventure|...|
 3175| 3.58| 75| Galaxy Quest (1999)|Adventure|Comedy|...|
 1088|3.369047619047619| 42|Dirty Dancing (1987)|Drama|Musical|Rom...|
 32460| 4.25| 4|Knockin' on Heave...|Action|Comedy|Cri...|
+-------+-----------------+-----+--------------------+--------------------+
only showing top 5 rows

### UserId order by rating count

In [74]:
ratings_df.groupBy("userId").count().orderBy('count',ascending=False).show(5)

+------+-----+
userId|count|
+------+-----+
 414| 2698|
 599| 2478|
 474| 2108|
 448| 1864|
 274| 1346|
+------+-----+
only showing top 5 rows

We will recommend user 414 and 599 as they have the highest rating count. This means the recommendations to them would be more reliable.

### Histogram of rating counts

In [77]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.distplot(ratings_info_df.toPandas()['count'],bins=50,ax = ax)
display()

### Histogram of the ratings

In [79]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.distplot(ratings_info_df.toPandas()['avg(rating)'],bins=50,ax=ax)
display()

In [80]:
fig, ax = plt.subplots()
sns.jointplot(x='avg(rating)', y='count', data=ratings_info_df.toPandas())
display()

## Recommend moive to users with id: 414, 599.

In [82]:
# use the recommendation function of ALS
ALS_recommendations = best_model.recommendForAllUsers(10)
ALS_recommendations.show(5)

+------+--------------------+
userId| recommendations|
+------+--------------------+
 471|[[6818, 5.00462],...|
 463|[[96004, 5.109864...|
 496|[[51931, 4.802244...|
 148|[[5034, 4.9191675...|
 540|[[96004, 5.485852...|
+------+--------------------+
only showing top 5 rows

We need to process the above dataframe for readability.

In [84]:
# Data procesing of the ALS_recommendations dataframe
recommendations_df = (ALS_recommendations\
                      .select("userId",\
                              F.explode("recommendations")\
                              .alias("recommendation"))\
                      .select("userId", "recommendation.movieId",\
                              F.col("recommendation.rating")\
                              .alias('prediction')))
recommendations_df.show(5)

+------+-------+----------+
userId|movieId|prediction|
+------+-------+----------+
 471| 6818| 5.00462|
 471| 8477| 5.004135|
 471| 3379| 4.8293505|
 471| 96004| 4.8293505|
 471| 25771| 4.7369065|
+------+-------+----------+
only showing top 5 rows

In [85]:
# we only recommend movies that have not been watched by users before
recommendations_df = recommendations_df.join(movies_df,["movieId"],"left").join(ratings_df,['movieId','UserId'],'left')
recommendations_df = recommendations_df.drop('timestamp')
recommendations_df = recommendations_df.filter(ratings_df.rating.isNull())
recommendations_df.show(5)

+-------+------+----------+--------------------+--------------------+------+
movieId|userId|prediction| title| genres|rating|
+-------+------+----------+--------------------+--------------------+------+
 6818| 471| 5.00462|Come and See (Idi...| Drama|War| null|
 8477| 471| 5.004135| Jetée, La (1962)| Romance|Sci-Fi| null|
 3379| 471| 4.8293505| On the Beach (1959)| Drama| null|
 96004| 471| 4.8293505|Dragon Ball Z: Th...|Action|Adventure|...| null|
 25771| 471| 4.7369065|Andalusian Dog, A...| Fantasy| null|
+-------+------+----------+--------------------+--------------------+------+
only showing top 5 rows

We will only recommend movies that the users haven't watched/rated.

movies recommendations for user 414

In [87]:
recommendations_df.filter(recommendations_df['userId'] == 414).show(5)

+-------+------+----------+--------------------+--------------------+------+
movieId|userId|prediction| title| genres|rating|
+-------+------+----------+--------------------+--------------------+------+
 8477| 414| 5.228586| Jetée, La (1962)| Romance|Sci-Fi| null|
 3379| 414| 5.190521| On the Beach (1959)| Drama| null|
 96004| 414| 5.190521|Dragon Ball Z: Th...|Action|Adventure|...| null|
 6818| 414| 5.1880026|Come and See (Idi...| Drama|War| null|
 102217| 414| 4.9642234|Bill Hicks: Revel...| Comedy| null|
+-------+------+----------+--------------------+--------------------+------+
only showing top 5 rows

movies recommendations for user 599

In [89]:
recommendations_df.filter(recommendations_df['userId'] == 599).show(5)

+-------+------+----------+--------------------+--------------------+------+
movieId|userId|prediction| title| genres|rating|
+-------+------+----------+--------------------+--------------------+------+
 33649| 599| 3.8415768| Saving Face (2004)|Comedy|Drama|Romance| null|
 60943| 599| 3.8278158| Frozen River (2008)| Drama| null|
 5915| 599| 3.8277671|Victory (a.k.a. E...| Action|Drama|War| null|
 5490| 599| 3.8277671| The Big Bus (1976)| Action|Comedy| null|
 132333| 599| 3.8277671| Seve (2014)| Documentary|Drama| null|
+-------+------+----------+--------------------+--------------------+------+
only showing top 5 rows

In [90]:
# Another way is to recommendForUserSubset function
#users = ALS_recommendations.filter(ALS_recommendations['userId'] == 575)
#ALS_recommendations_target = best_model.recommendForUserSubset(users,1)
#ALS_recommendations_target.show()

## Find the similar moives for moive with id: 464, 471

The similarites of different movies can be recognized from correlations of their user ratings. Let's say user A and B both gave movie 1,2 5 star rating. This indicates that the movie 1 and 2 might be highily similar.

However, we have a challenge in that some of the movies have very few ratings and may end up having high correlation simply because one or two people gave them a 5 star rating. We can fix this by setting a threshold for the number of ratings. From the histogram earlier we saw a sharp decline in number of ratings from 100. Therefore we will choose this as our threshold.

In [93]:
movie_matrix = movie_ratings_df.toPandas().pivot_table(index='userId', columns='movieId', values='rating')

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [94]:
def find_similar_movie(x):
  movie_x_rating = movie_matrix[x]
  similar_to_x=movie_matrix.corrwith(movie_x_rating).reset_index(level=0)
  similar_to_x.dropna(axis = 0,how = 'any',inplace=True)
  similar_to_x.columns = ['movieId','correlation']
  
  similar_to_x_df = sqlContext.createDataFrame(similar_to_x)
  similar_to_x_movie = similar_to_x_df.join(ratings_info_df,'movieId','left').toPandas()[['movieId','correlation','title','count']]
  res = similar_to_x_movie[similar_to_x_movie['count']>100].sort_values(by = 'correlation',ascending = False)
  return similar_to_x,res

We will only find similar movies that have been rated more than 100 times.

movies similar to movie 471

In [96]:
# Movies similar to 471
corr_471,similar_to_471_movie = find_similar_movie(471)

/databricks/python/lib/python3.7/site-packages/numpy/lib/function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
 c = cov(x, y, rowvar)
/databricks/python/lib/python3.7/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
 c *= np.true_divide(1, fact)
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [97]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.scatterplot(x = 'movieId',y='correlation',data=corr_471,ax=ax)
display()

In [98]:
spark.createDataFrame(similar_to_471_movie).show(5)

+-------+------------------+--------------------+-----+
movieId| correlation| title|count|
+-------+------------------+--------------------+-----+
 79132|0.9731236802019037| Inception (2010)| 143|
 48516|0.7248611789946814|Departed, The (2006)| 107|
 33794|0.6774296636907189|Batman Begins (2005)| 116|
 1222|0.6081008740813361|Full Metal Jacket...| 102|
 364|0.5705494421677838|Lion King, The (1...| 172|
+-------+------------------+--------------------+-----+
only showing top 5 rows

movies similar to movie 464

In [100]:
# Movies similar to 464
corr_464,similar_to_464_movie = find_similar_movie(464)

/databricks/python/lib/python3.7/site-packages/numpy/lib/function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
 c = cov(x, y, rowvar)
/databricks/python/lib/python3.7/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
 c *= np.true_divide(1, fact)
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [101]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.scatterplot(x='movieId',y='correlation',data=corr_464,ax=ax)
display()

In [102]:
spark.createDataFrame(similar_to_464_movie).show(5)

+-------+------------------+--------------------+-----+
movieId| correlation| title|count|
+-------+------------------+--------------------+-----+
 5816| 1.0|Harry Potter and ...| 102|
 4896|0.9045340337332909|Harry Potter and ...| 107|
 58559|0.8660254037844387|Dark Knight, The ...| 149|
 253|0.8539125638299664|Interview with th...| 109|
 7153|0.8451542547285166|Lord of the Rings...| 185|
+-------+------------------+--------------------+-----+
only showing top 5 rows

## Report
In this project, the movie data from movie lens data set which includes about 600 users and 9500 movies were analyzed to gain insights on movie recomendations to users and finding similar movies. At the beginning, we calculate the sparisity of the movie ratings which is 98.3%, this tells us that the ratings dataframe is mostly empty, which brings significance to predict the user ratings from what we have.

To achieve the goal,the data was analyzed on Spark platform from perfoming data cleaning,processing to model training with Alternating Least Squares (ALS) algorithm.During which, grid search and cross validation were applied to tune the hyperparameters. It is found that using large rank and iterzations would help achieve a low RMSE. Finally, we choose a rank of 10, iterate 50 times and regPram = 0.15,a RMSE of 0.69 was achieved.This means that on average the model predicts 0.69 above or below values of the original ratings matrix.

By successsfully predicting the ratings using the best model, we not only fill the rating dataframe and recommend our users with movies they have never watched, but also find similar movies through their correlations. This brings huge business value to the company.